In [8]:
!pip install ucimlrepo
!pip install mlflow

In [9]:
# from ucimlrepo import fetch_ucirepo

# # fetch dataset
# df = fetch_ucirepo(id=45)

# # data (as pandas dataframes)
# X = df.data.features
# y = df.data.targets

# # metadata
# print(df.metadata)

# # variable information
# print(df.variables)


In [10]:
# # Display the head of the features DataFrame
# print("Head of features:")
# print(df.data.features.head())
# print(df.isnull().sum())

# # Display the head of the targets DataFrame (optional)
# # print("\nHead of targets:")
# # print(heart_disease.data.targets.head())

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn

In [12]:
import pandas as pd

# Replace 'heart.csv' with the actual filename if different
df = pd.read_csv('data.csv')

print(df.head())
print(df.info())
print(df.isnull().sum())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

    ca  thal  num  
0  0.0   6.0    0  
1  3.0   3.0    2  
2  2.0   7.0    1  
3  0.0   3.0    0  
4  0.0   3.0    0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       

In [13]:
# import pandas as pd

# Define column names based on UCI Heart Disease dataset
column_names = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'
]

# Load the CSV file into a DataFrame
df = pd.read_csv('data.csv', names=column_names)

In [14]:
# import numpy as np

# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

# Convert relevant columns to numeric
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with any missing values
df.dropna(inplace=True)

# Reset index after dropping
df.reset_index(drop=True, inplace=True)

In [15]:
# Cast appropriate columns to int
int_cols = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
            'thalach', 'exang', 'slope', 'ca', 'thal', 'target']
for col in int_cols:
    df[col] = df[col].astype(int)

# 'oldpeak' is float
df['oldpeak'] = df['oldpeak'].astype(float)

In [16]:
# Ensure target is binary
df['target'] = df['target'].apply(lambda x: 1 if x > 0 else 0)

In [17]:
print("First 5 Rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nMissing Values per Column:")
print(df.isnull().sum())

First 5 Rows:
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

   ca  thal  target  
0   0     6       0  
1   3     3       1  
2   2     7       1  
3   0     3       0  
4   0     3       0  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       297 non-null    int64  
 1   sex       297 non-null    int64  
 2   cp        297 non-null    int64  
 3   trestbps  297 non-null    int64  
 4   chol    

In [18]:
# 6. Define features X and target y
X = df.drop('target', axis=1)   # All rows, all columns except 'target'
y = df['target']                # Target column

# 7. Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Preserve class distribution
)

# 8. Print shapes for confirmation
print("\nData shapes:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


Data shapes:
X_train: (237, 13), X_test: (60, 13)
y_train: (237,), y_test: (60,)


In [19]:
# Identify numerical and categorical columns
numerical_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(exclude=np.number).columns

# Define transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Define full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index([], dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict
y_pred = pipeline.predict(X_test)

# Calculate metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8667
Precision: 0.8846
Recall: 0.8214
F1 Score: 0.8519


In [22]:
import subprocess
import atexit

# Start MLflow UI on port 9000
mlflow_process = subprocess.Popen(["mlflow", "ui", "--port", "9000"])
atexit.register(mlflow_process.terminate)

print("MLflow UI started on port 9000")

MLflow UI started on port 9000


In [23]:
!pip install pyngrok

In [24]:
from pyngrok import ngrok

# Replace '<your-authtoken>' with your actual token
ngrok.set_auth_token("2xFxwGCmYLD1qlmyuZRacOkUTfL_xJduyAA1wMQSND6C5FgY")

In [25]:
import subprocess

# Start MLflow UI on port 9000
mlflow_process = subprocess.Popen(["mlflow", "ui", "--port", "9000"])

In [26]:
# Connect port 9000 to public URL
public_url = ngrok.connect(9000)
print(f"MLflow UI is available at: {public_url}")

ERROR:pyngrok.process.ngrok:t=2025-05-19T06:21:44+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
# Step 1: Install dependencies (if not done already)
# !pip install mlflow pyngrok

# Step 2: Load libraries
from pyngrok import ngrok
import subprocess

# Step 3: Set your ngrok authtoken (replace with yours)
ngrok.set_auth_token("2xFxwGCmYLD1qlmyuZRacOkUTfL_xJduyAA1wMQSND6C5FgY")  # ← Replace this!

# Step 4: Start MLflow UI
mlflow_process = subprocess.Popen(["mlflow", "ui", "--port", "9000"])

# Step 5: Create ngrok tunnel
public_url = ngrok.connect(9000)
print(f"MLflow UI is now available at: {public_url}")

In [ ]:
from pyngrok import ngrok

# Terminate any existing tunnels
ngrok.kill()

# Start tunnel for port 9000
public_url = ngrok.connect(9000)
print(f"MLflow UI is available at: {public_url}")

In [27]:
import mlflow
import mlflow.sklearn

# Set tracking URI
mlflow.set_tracking_uri("file:./mlruns")

# Set experiment name
mlflow.set_experiment("Heart Disease Prediction RF")

# Start run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("model_type", "RandomForest")

    # Log metrics
    mlflow.log_metric("Accuracy", acc)
    mlflow.log_metric("Precision", prec)
    mlflow.log_metric("Recall", rec)
    mlflow.log_metric("F1 Score", f1)

    # Log model
    mlflow.sklearn.log_model(pipeline, "heart_disease_model")

    print("Model logged under Run ID:", mlflow.active_run().info.run_id)

2025/05/19 06:22:37 INFO mlflow.tracking.fluent: Experiment with name 'Heart Disease Prediction RF' does not exist. Creating a new experiment.
2025/05/19 06:22:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged under Run ID: 7c64835d45ea4cb792701bda467de0f2
